# Julia é rápido

É muito comum usar _benchmarks_ para comparar linguagens.  Estes _benchmarks_ geram longas discussões referentes ao que está sendo testado e como explicar as diferenças observadas.  Estas questões simples podem ser muito mais complicadas do que parece inicialmente.

O propósito deste notebook é você ver como se faz um _benchmark_. .

(Este material começou como uma palestra excelente de  Steven Johnson no MIT: https://github.com/stevengj/18S096/blob/master/lectures/lecture1/Boxes-and-registers.ipynb.)

# Resumo deste notebook

- Definir a função soma
- Implementar e testar a função em ...
    - C (escrito a mão)
    - C (escrito a mão com -ffast-math)
    - python (biblioteca padrão)
    - python (numpy)
    - python (escrito a mão)
    - Julia (biblioteca padrão)
    - Julia (escrito a mão)
    - Julia (escrito a mão com SIMD)
- Resumo dos resultados obtidos

# `sum`: Função simples de entender

Considere a função **sum**  `sum(a)`, que calcula 
$$
\mathrm{sum}(a) = \sum_{i=1}^n a_i,
$$
onde $n$ é  comprimento de  `a`.

In [ ]:
a = rand(10^7) # Vetor 1D vector de números aleatórios, uniforme em  [0,1)

In [ ]:
sum(a)

O valor esperado é 0.5 * 10^7, já que a média de cada elemento é 0.5

# Benchmarking de diferentes maneiras em diferentes  languages

In [ ]:
@time sum(a)

@time sum(a)

@time sum(a)

A macro `@time` pode resultar em resultados com ruído então não é a melhor escolha para fazer o benchmark!

Por sorte, Julia tem o pacote `BenchmarkTools.jl` para fazer benchmarks confiáveis de maneira simples:

In [ ]:
# using Pkg
# Pkg.add("BenchmarkTools")

In [ ]:
using BenchmarkTools  

#  1. A linguagem C 

C é muitas vezes considerado o adversário a ser vencido: difícil para as pessoas, bom para a máquina. Chegar a um fator de 2 dos resultados obtidos com C é algo geralmente positivo. Mas mesmo assim, mesmo trabalhando com C, existem muitas otimizações possíveis que o programador pode ou não utilizar.

O código em C abaixo vai ser utilizado e compilado a partir deste notebook.

In [ ]:
using Libdl
C_code = """
#include <stddef.h>
double c_sum(size_t n, double *X) {
    double s = 0.0;
    for (size_t i = 0; i < n; ++i) {
        s += X[i];
    }
    return s;
}
"""

const Clib = tempname()   # make a temporary file


# compile to a shared library by piping C_code to gcc
# (works only if you have gcc installed):

open(`gcc -fPIC -O3 -msse3 -xc -shared -o $(Clib * "." * Libdl.dlext) -`, "w") do f
    print(f, C_code) 
end

# define a Julia function that calls the C function:
c_sum(X::Array{Float64}) = ccall(("c_sum", Clib), Float64, (Csize_t, Ptr{Float64}), length(X), X)

In [ ]:
c_sum(a)

In [ ]:
c_sum(a) ≈ sum(a) # type \approx and then <TAB> to get the ≈ symbolb

In [ ]:
c_sum(a) - sum(a)  

In [ ]:
≈  # alias for the `isapprox` function

In [ ]:
?isapprox

Agora podemos fazer o benchmark do código em  C diretamente da Julia:

In [ ]:
c_bench = @benchmark c_sum($a)

In [ ]:
println("C: Fastest time was $(minimum(c_bench.times) / 1e6) msec")

In [ ]:
d = Dict()  # a "dictionary", i.e. an associative array
d["C"] = minimum(c_bench.times) / 1e6  # in milliseconds
d

In [ ]:
using Plots
gr()

In [ ]:
using Statistics # bring in statistical support for standard deviations
t = c_bench.times / 1e6 # times in milliseconds
m, σ = minimum(t), std(t)

histogram(t, bins=500,
    xlim=(m - 0.01, m + σ),
    xlabel="milliseconds", ylabel="count", label="")

# 2. C com -ffast-math

Se permitirmos que C rearrange as operações de ponto flutuante, então o código pode ser vetorizado usando instruções SIMD (_single instruction, multiple data_ - Instrução única, dados múltiplos) .

In [ ]:
const Clib_fastmath = tempname()   # make a temporary file

# The same as above but with a -ffast-math flag added
open(`gcc -fPIC -O3 -msse3 -xc -shared -ffast-math -o $(Clib_fastmath * "." * Libdl.dlext) -`, "w") do f
    print(f, C_code) 
end

# define a Julia function that calls the C function:
c_sum_fastmath(X::Array{Float64}) = ccall(("c_sum", Clib_fastmath), Float64, (Csize_t, Ptr{Float64}), length(X), X)

In [ ]:
c_fastmath_bench = @benchmark $c_sum_fastmath($a)

In [ ]:
d["C -ffast-math"] = minimum(c_fastmath_bench.times) / 1e6  # in milliseconds

# 3. Usando a função `sum` do Python

O pacote  `PyCall` fornece uma interface para Python a partir da Julia:

In [ ]:
# using Pkg; Pkg.add("PyCall")
using PyCall

In [ ]:
# get the Python built-in "sum" function:
pysum = pybuiltin("sum")

In [ ]:
pysum(a)

In [ ]:
pysum(a) ≈ sum(a)

In [ ]:
py_list_bench = @benchmark $pysum($a)

In [ ]:
d["Python built-in"] = minimum(py_list_bench.times) / 1e6
d

# 4. Python: `numpy` 

## Usa os recursos "SIMD" do hardware mas só funciona quando funciona.

`numpy` é uma biblioteca otimizada C que é chamada do Python. Pode ser instalado na Julia como mostrado a seguir:

In [ ]:
# using Pkg; Pkg.add("Conda")
using Conda

In [ ]:
# Conda.add("numpy")

In [ ]:
numpy_sum = pyimport("numpy")["sum"]

py_numpy_bench = @benchmark $numpy_sum($a)

In [ ]:
numpy_sum(a)

In [ ]:
numpy_sum(a) ≈ sum(a)

In [ ]:
d["Python numpy"] = minimum(py_numpy_bench.times) / 1e6
d

# 5. Python, escrito a mão

In [ ]:
py"""
def py_sum(A):
    s = 0.0
    for a in A:
        s += a
    return s
"""

sum_py = py"py_sum"

In [ ]:
py_hand = @benchmark $sum_py($a)

In [ ]:
sum_py(a)

In [ ]:
sum_py(a) ≈ sum(a)

In [ ]:
d["Python hand-written"] = minimum(py_hand.times) / 1e6
d

# 6. Julia (biblioteca padrão) 

## Escrito em Julia, não C!

In [ ]:
@which sum(a)

In [ ]:
j_bench = @benchmark sum($a)

In [ ]:
d["Julia built-in"] = minimum(j_bench.times) / 1e6
d

# 7. Julia (escrito a mão) 

In [ ]:
function mysum(A)   
    s = 0.0 # s = zero(eltype(a))
    for a in A
        s += a
    end
    s
end

In [ ]:
j_bench_hand = @benchmark mysum($a)

In [ ]:
d["Julia hand-written"] = minimum(j_bench_hand.times) / 1e6
d

# 8. Julia (escrito a mão com simd) 

In [ ]:
function mysum_simd(A)   
    s = 0.0 # s = zero(eltype(A))
    @simd for a in A
        s += a
    end
    s
end

In [ ]:
j_bench_hand_simd = @benchmark mysum_simd($a)

In [ ]:
mysum_simd(a)

In [ ]:
d["Julia hand-written simd"] = minimum(j_bench_hand_simd.times) / 1e6
d

# Resumo

In [ ]:
for (key, value) in sort(collect(d), by=last)
    println(rpad(key, 25, "."), lpad(round(value; digits=1), 6, "."))
end